In [2]:
import pandas as pd
import openai
import uuid
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import os
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('./Data/Google Reviews.csv')  ## Google review dataset
#df = pd.read_csv('./Data/Play Store App reviews.csv')  ## App reviews from Google Play Storedf = df[['Review_Text','Final Coding']]
X_data = df[['Review_Text']]
y_data = df[['Final Coding']]

In [4]:
rus = RandomOverSampler(random_state= 42)
x_rus ,y_rus = rus.fit_resample(X_data,y_data)
X_train, X_test, y_train, y_test = train_test_split(x_rus, y_rus, test_size=0.3, random_state=777)


In [4]:
train_df = pd.DataFrame(X_train)
train_df['completion'] = y_train

eval_df = pd.DataFrame(X_test)
eval_df['completion'] = y_test

train_df.rename(columns={'Review_Text':'prompt'},inplace=1)
eval_df.rename(columns={'Review_Text':'prompt'},inplace=1)


In [3]:
train_df.to_json("train.jsonl",orient='records',lines=True)

In [1]:
!openai tools fine_tunes.prepare_data -f train.jsonl -q    

In [2]:

!openai -k "Your_API_KEY" api fine_tunes.create -t "training_dataset_oversample_google_prepared.jsonl" --model ada

In [3]:
!openai -k "Your_API_KEY" api fine_tunes.follow -i ft-M6OzoH80NLjn8pLCmv7WEOl8


In [3]:
gpt_predict = []
ft_model = 'Uploaded model name enter here'

for row,item in test.iterrows():
    res = openai.Completion.create(model= ft_model,prompt=item['prompt'],max_tokens=1,temperature=0 )
    gpt_predict.append(res['choices'][0]['text'])

In [2]:
y_pred = [1 if x == ' 1' else 0 for x in gpt_predict]
y_pred

In [1]:
print("Accuracy is ", accuracy_score(test_df['completion'],y_pred))
print("Precision is ",precision_score(test_df['completion'],y_pred))
print("Recall is ",recall_score(test_df['completion'],y_pred))
print("F1-score is",f1_score(test_df['completion'],y_pred))
print("Mathew corrcoef is ",matthews_corrcoef(test_df['completion'],y_pred))